In [ ]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
#load processed and resampled train dataset
df = pd.read_csv('processed/train_dataset.csv')

In [ ]:
import numpy as np

# Conditions
conditions = [
    (df['ind_recommended'] == 1) & (df['activation'] == 1),  # both are 1
    (df['ind_recommended'] == 1) & (df['activation'] == 0),  # ind_recommended is 1, activation is 0
    (df['ind_recommended'] == 0) & (df['activation'] == 1),  # ind_recommended is 0, activation is 1
    (df['ind_recommended'] == 0) & (df['activation'] == 0)   # both are 0
]

# Corresponding scores for each condition
scores = [
    1,    # Score for condition 1
    0,    # Score for condition 2
    0,    # Score for condition 3
    1   # Score for condition 4
]

df['score'] = np.select(conditions, scores)
df.drop(['activation','ind_recommended'], axis=1, inplace = True)

y = df_train['score']
df_train.drop(['merchant','customer','score'], axis=1, inplace=True)

X_train, y_train = df_train, y

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'max_depth': [6, 8, 10, 12],
    'learning_rate': [0.1, 0.05, 0.01],
    'n_estimators': [100, 200, 500],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
    'lambda': [1e-5, 1e-2, 0.1, 1],
    'alpha': [1e-5, 1e-2, 0.1, 1]
}

# Initialize the XGBRegressor
xgb_reg = XGBRegressor(use_label_encoder=False, eval_metric='rmse', tree_method='hist', device='cuda')

# Setup RandomizedSearchCV
random_cv = RandomizedSearchCV(xgb_reg, param_grid, n_iter=25, scoring='neg_mean_squared_error', cv=3, n_jobs=-1, verbose=1, random_state=42)

# Fit RandomizedSearchCV
random_cv.fit(X_train, y_train)

# Print best parameters
print(f"Best parameters: {random_cv.best_params_}")

In [ ]:
# Initialize XGBoost classifier with recommended starting hyperparameters
xgb_reg = XGBRegressor(
    use_label_encoder=False,
    eval_metric='rmse',
    tree_method='hist',
    max_depth=12,
    learning_rate=0.05,
    n_estimators=500,
    subsample=0.8,
    colsample_bytree=0.8,
    alpha=0.01,
    reg_lambda=1,
    gamma=0,
    random_state=42
)
# Fit the model to the training data
xgb_reg.fit(X_train, y_train)